# SWD 6: High Performance Python

## Benchmarking

### Time

Use `timeit` magic tool.

- In a function: `%timeit function()`
- In a code cell: `%%timeit`

In [1]:
# function
%timeit list(range(100))

1.52 µs ± 505 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [2]:
# Code Cell
%%timeit

for i in range(1_000_000):
    pass

29.5 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Profiler

- tool: `line_profiler`
- Colab setup steps:
    1. Install `line_profiler`
    2. load the magic tool
- call the tool using: `%lprun -f function_name function_name(function_parameters)`

In [3]:
# Install `line_profiler`
!pip install --quiet line_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 10.1 MB/s eta 0:00:00


In [4]:
# Load line_profiler
%load_ext line_profiler

In [5]:
# Define some functions to profile
import time

# Function that runs quickly
def relax():
    pass

# Function that spend a long time to run
def bottleneck():
    time.sleep(0.001)

# Function that do some operations and call other functions
def some_function():
    nums = [num for num in range(1_000)] #list comprehension
    bigger_nums = [num**2 for num in nums]
    bottleneck()
    print("hello world")
    for i in range(10_000):
        relax()

Can you identify the part of the code responsible for making it inefficient?

In [ ]:
# Run the linear_profiler
%lprun -f some_function some_function()

## Data Structures

### Built-in data structures

- Create a list with numbers between 0 and 10000.
- Create a second list where every item is the correspondent item in list1 but multiplied by 2
- Use `timeit` to measure the time your code cell needs to run

In [7]:
%%timeit

list1 = list(range(10_000))

# list1*2 >> This approach is wrong

# The same as the above, but without list comprehension syntax:
list2 = []
for item in list1:
   list2.append(item*2)

1.85 ms ± 726 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We can improve the performance by replacing the loop with a list comprehension syntax:

In [8]:
%%timeit

list1 = list(range(10_000))
list2 =[item*2 for item in list1]

1.36 ms ± 687 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### External data structures

- Import `numpy` and repeat the above, but using arrays instead lists

In [9]:
import numpy as np

In [10]:
%%timeit

list1 = list(range(10_000)) # create a list
array1 = np.array(list1) # transform list to numpy array
array2 = array1*2

2.58 ms ± 442 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The performance is worse.
This is because we spend a long time converting the list to an array.

One better alternative is to create the array from scratch instead convert from list.

In [11]:
%%timeit

array1 = np.arange(10_000)  # create an array
array2 = array1*2

26.1 µs ± 6.28 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Homemade Functions

Create a function to calculate the hypotenuse, i.e.

$$
hypot = \sqrt{(a^2 + b^2)}
$$

In [12]:
def hypotenuse(x,y):
    return (x**2 + y**2)**0.5

In [13]:
%timeit hypotenuse(3,4)

824 ns ± 182 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


We can try to improve by using appropriated functions, like pow for power operations:

In [14]:
def hypotenuse2(x,y):
    return (pow(x,2) + pow(y,2))**0.5

In [15]:
%timeit hypotenuse2(3,4)

717 ns ± 6.16 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


We can try to improve by using mathematical knowledge and choosing different/equivalent functions, for example:

$$
3^2 = 3\times3
$$

For python, multiplications are faster than power operations.

In [16]:
def hypotenuse3(x,y):
    return (x*x + y*y)**0.5

In [17]:
%timeit hypotenuse3(3,4)

210 ns ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### External Functions

External functions are optimised, and, usually, compiled. So, they tend to be more efficient, even for a simple function.

The package `math` has a function to calculate the hypotenuse:

In [18]:
import math

In [19]:
%timeit math.hypot(3,4)

150 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#### Tip: AVOID DOT OPERATION

If you import directly the function that you need instead the entire package, the result is going to be a little more efficient.

In [20]:
from math import hypot

In [21]:
%timeit hypot(3,4)

127 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Broadcasting

Broadcasting is the process that happens in the background in vectorised operations. It is an attempt to perform operations even when the arrays does not have the same shape.

Note: this is not always possible, see the example bellow:

In [2]:
import numpy as np

In [3]:
col_array  = np.array([0, 10, 20, 30])
row_array = np.array([0, 1, 2])

In [4]:
col_array + row_array

ValueError: operands could not be broadcast together with shapes (4,) (3,) 

We can not sum two line arrays with different sizes, but we can do it if one of the arrays is a column array:

In [24]:
col_array  = np.array([0, 10, 20, 30]).reshape(4, 1)
row_array = np.array([0, 1, 2])

In [25]:
col_array

array([[ 0],
       [10],
       [20],
       [30]])

In [26]:
row_array

array([0, 1, 2])

In [27]:
col_array + row_array

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

## Vectorisation

The advantage of arrays over lists, is that arrays are vectorised. Using the following approach you can also vectorise your functions.

Lets first create a function. In this case, lets try the gaussian distribution function that looks like:

$$
f(x) = \dfrac{1}{\sigma\sqrt{2\pi}} e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma} \right)^2}
$$

where: $\mu$ is the mean and $\sigma$ is the standard deviation.

In [28]:
def gauss(x, mean, sigma):
    A = 1.0 / (sigma*np.sqrt(2.0*np.pi))
    B = ((x - mean) / sigma)**2
    return A*np.exp(-0.5*B)

In [29]:
mean = 0
sigma = 1

In [30]:
gauss(50, mean, sigma)

0.0

Now create a big list around zero

In [31]:
x = list(range(-10_000,10_000, 1))


Following, we are going to get the result of this function for every x-value inside our list

### Item approach

For comparison, lets first use the item approach

In [32]:
%%timeit
y = []
for i in x:
    y.append(gauss(i, mean, sigma))

76.6 ms ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
# import matplotlib.pyplot as plt
# plt.plot(x,y)

### Vectorise approach

In [34]:
vector_gauss = np.vectorize(gauss)

In [35]:
%%timeit
y = vector_gauss(x, mean, sigma)

58.5 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
# plt.plot(x,y)

### Be smart!

In this case, all operations inside our function were already vectorised (using `numpy`), so we could use an array as input and avoid the vectorisation step.

In [37]:
%%timeit
x = np.arange(-10_000,10_000, 1)
gauss(x, mean, sigma)

445 µs ± 26.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Compiler

Lets create our own compiled functions. To do this we can use `numba` and the decorator @njit (or @jit)


In [38]:
from numba import njit # non-python mode # faster
from numba import jit # object mode

First lets define one arbitrary function

In [39]:
def slow_func(nums):
    trace = 0.0
    for num in nums:
        trace = trace + np.cos(num)
    result = nums + trace
    return result

Now, lets create an array to test the function

In [40]:
nums = np.arange(1_000_000)

Using the standard approach, the time is:

In [41]:
%timeit slow_func(nums)

2 s ± 419 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To compile, simple add the decorator above the function:

In [42]:
@njit # decorator
def fast_func(nums):
    trace = 0.0
    for num in nums:
        trace = trace + np.cos(num)
    result = nums + trace
    return result

In [43]:
%timeit fast_func(nums)

24.2 ms ± 981 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Parallelisation

### Numba parallelisation

`Numba` has a lot of functionalities beyond the compiler. One is related with parallelisation.

Lets first create a standard function and an array to use as input:

In [44]:
def my_serial_function(x):
    return np.cos(x)**2 + np.sin(x)**2

In [45]:
x = np.arange(1_000_000, dtype=np.int64)

In [46]:
%timeit my_serial_function(x)

51 ms ± 8.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Now, lets parallelise using `numba` by simple adding the option parallel in the compiler decorator.

In [47]:
@njit(parallel=True)
def my_parallel_function(x):
    return np.cos(x)**2 + np.sin(x)**2

In [48]:
my_parallel_function(x)

array([1., 1., 1., ..., 1., 1., 1.])

In [49]:
%timeit my_parallel_function(x)

28.1 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Although simple, this does not give us control about the parallelisation process. The next tool is the best tool we know to control and manage the different tasks in a parallelisation process

### Dask

First we are going to create a huge operation that is not even possible to execute in colab.

In [50]:
import numpy as np

In [51]:
# The next is going to break the runtime, because needs to much memory
# x = np.random.random(size=(10_000, 10_000, 10))
# y = np.random.random(size=(10_000, 10_000, 10))
# z = (np.arcsin(x) + np.arccos(y)).sum(axis=(0,1,2))

Now, lets configure dask:

In [52]:
from dask.distributed import Client, LocalCluster

In [53]:
cluster = LocalCluster()
client = Client()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:34537
INFO:distributed.scheduler:  dashboard at:            127.0.0.1:8787
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43539'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45129'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:39267', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39267
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:40216
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:46551', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:46551
INFO:distributed.core:Startin

Above we have some information about the client, including an address to see the dashboard (127.0.0.1:8787). Unfortunately this dashboard does not work with Colab.

You can get some additional information open the client:

In [54]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34877/status,
Dashboard: http://127.0.0.1:34877/status,Workers: 2
Total threads: 2,Total memory: 12.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34387,Workers: 2
Dashboard: http://127.0.0.1:34877/status,Total threads: 2
Started: Just now,Total memory: 12.68 GiB
Comm: tcp://127.0.0.1:41685,Total threads: 1
Dashboard: http://127.0.0.1:43773/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:38363,


Now we need rewrite the function using `dask`. `Dask` is compatible with some packages, including `Numpy` via `dask.array`.

So we need to copy the function and change `np.` by `da.`

In [55]:
import dask.array as da

In [56]:
# crtl+d
x = da.random.random(size=(10_000, 10_000, 10))
y = da.random.random(size=(10_000, 10_000, 10))
z = (da.arcsin(x) + da.arccos(y)).sum(axis=(0,1))

Note that the code above does not perform any calculation, it is just to organise the structure of the problem.

You can see how the output is going to looks like:

In [57]:
z

dask.array<sum-aggregate, shape=(10,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>

You can also see how the entire computational process is going to be performed:

In [58]:
z.visualize()

*Note: this image is automatically saved as `mydask.png`*

Finally, you can compute the result

In [59]:
result = z.compute()

In [60]:
result

array([1.57082115e+08, 1.57076627e+08, 1.57075394e+08, 1.57078165e+08,
       1.57092538e+08, 1.57078882e+08, 1.57082231e+08, 1.57074016e+08,
       1.57074400e+08, 1.57085177e+08])

#### Report

If you want a report, you can do this via `performance_report`:

In [61]:
from dask.distributed import performance_report

In [62]:
with performance_report(filename="dask-report.html"):
    result = z.compute()

INFO:distributed.worker:Run out-of-band function 'lambda'


## GPU

To run in a GPU we need to change the runtime type:

Runtime >> Change Runtime type >> Hardware accelerator >> GPU

Verify what card you got:

In [1]:
!nvidia-smi

Wed Jun 28 12:28:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

*Note: If this code fails, it is probably because GPU was not loaded correctly. Be sure that you changed your runtime.*

Define the function and the input:

In [4]:
import numpy as np

In [65]:
x = np.arange(1.0e7, dtype=np.float32)

In [66]:
def my_serial_function_for_cpu(x):
    return np.cos(x)**2 + np.sin(x)**2

In [67]:
%timeit my_serial_function_for_cpu(x)

283 ms ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### CPU compiled approach

For comparison, lets first try the compiled approach:

In [2]:
from numba import njit

In [69]:
@njit
def my_serial_compiled_function_for_cpu(x):
    return np.cos(x)**2 + np.sin(x)**2

In [70]:
%timeit my_serial_compiled_function_for_cpu(x)

160 ms ± 5.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### GPU Approach

Finally lets use the GPU. But needs some tweaks:

- GPU works better with float32
- To use GPU we need to specify the input type and the output type during the compilation
- We need to use `cuda` compatible functions. For instance `numpy.cos()` and `numpy.sin()` are not allawed. So lets use `math.cos()` and `math.sin()`

In [3]:
from numba import vectorize, float32
import math

In [5]:
x = np.arange(1.0e7, dtype=np.float32)

In [6]:
@vectorize(["float32(float32)"], target="cuda")
def my_serial_function_for_gpu(x):
    return math.cos(x)**2 + math.sin(x)**2

In [7]:
%timeit my_serial_function_for_gpu(x)

29.3 ms ± 4.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
